# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,25.99,83,75,5.14,CC,1731518813
1,1,port-aux-francais,-49.3500,70.2167,3.74,90,100,12.13,TF,1731518815
2,2,jamestown,42.0970,-79.2353,8.90,41,0,2.68,US,1731518731
3,3,badger,64.8000,-147.5333,-17.88,78,75,1.54,US,1731518817
4,4,iskateley,67.6803,53.1512,-1.82,98,100,4.39,RU,1731518818


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True, 
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800, 
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[(city_data_df['Max Temp'] <= 30) & (city_data_df['Max Temp'] >= 18) & (city_data_df["Humidity"] <70) & (city_data_df["Cloudiness"] <80)]
print(len(ideal_df))
# Drop any rows with null values
ideal_df.dropna()
# Display sample data
ideal_df.head()

90


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,zemio,5.0314,25.1361,25.82,69,32,2.37,CF,1731518877
55,55,tumayr,25.7039,45.8680,22.31,28,4,4.35,SA,1731518879
62,62,ujae,8.9322,165.7642,28.72,69,52,7.82,MH,1731518887
76,76,sur,22.5667,59.5289,27.37,59,10,2.48,OM,1731518757
85,85,cabo san lucas,22.8909,-109.9124,29.17,45,0,3.13,MX,1731518917


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
53,zemio,CF,5.0314,25.1361,69,
55,tumayr,SA,25.7039,45.8680,28,
62,ujae,MH,8.9322,165.7642,69,
76,sur,OM,22.5667,59.5289,59,
85,cabo san lucas,MX,22.8909,-109.9124,45,
...,...,...,...,...,...,...
544,touros,BR,-5.1989,-35.4608,60,
546,rongelap,MH,11.1635,166.8941,67,
548,unye,NG,11.4502,13.9620,24,
550,letlhakeng,BW,-24.0944,25.0298,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "categories":'accommodation.hotel',
    "limit":1, 
    "filter":'',
    "apiKey": geoapify_key,
    "bias":''
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    long = hotel_df.loc[index, 'Lng']
    lat = hotel_df.loc[index, 'Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{long},{lat},{radius}'
    params["bias"] = f'proximity:{long},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # No hotels were being found use the following to trouble shoot
    print(name_address)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
<Response [200]>
zemio - nearest hotel: No hotel found
<Response [200]>
tumayr - nearest hotel: No hotel found
<Response [200]>
ujae - nearest hotel: No hotel found
<Response [200]>
sur - nearest hotel: Sur Hotel
<Response [200]>
cabo san lucas - nearest hotel: Comfort Rooms
<Response [200]>
teknaf - nearest hotel: Sea View Hotel and Sports
<Response [200]>
fortuna foothills - nearest hotel: Comfort Inn
<Response [200]>
mount isa - nearest hotel: Ibis Styles
<Response [200]>
necochea - nearest hotel: Cabañas Río Quequén
<Response [200]>
san patricio - nearest hotel: Best Western Plus
<Response [200]>
as sulayyil - nearest hotel: No hotel found
<Response [200]>
dikhil - nearest hotel: No hotel found
<Response [200]>
mahad - nearest hotel: Sagar Hotel
<Response [200]>
mar del plata - nearest hotel: Hotel 3 de Abril
<Response [200]>
hadibu - nearest hotel: No hotel found
<Response [200]>
kudahuvadhoo - nearest hotel: Hotel Riu Palace Maldivas
<Response [200]>
santa r

,City,Country,Lat,Lng,Humidity,Hotel Name
53,zemio,CF,5.0314,25.1361,69,No hotel found
55,tumayr,SA,25.7039,45.8680,28,No hotel found
62,ujae,MH,8.9322,165.7642,69,No hotel found
76,sur,OM,22.5667,59.5289,59,Sur Hotel
85,cabo san lucas,MX,22.8909,-109.9124,45,Comfort Rooms
...,...,...,...,...,...,...
544,touros,BR,-5.1989,-35.4608,60,Pousada Atlântico
546,rongelap,MH,11.1635,166.8941,67,No hotel found
548,unye,NG,11.4502,13.9620,24,No hotel found
550,letlhakeng,BW,-24.0944,25.0298,29,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    fame_width = 700,
    frame_height = 400,
    color = 'City',
    alpha=0.6,
    hover_cols=['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel Name'],
    #use_index=False,
    size="Humidity",
    scale = 1
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)